In [1]:
# Styling notebook
from IPython.core.display import HTML
def css_styling():
    styles = open("rise.css", "r").read()
    return HTML(styles)
css_styling()

<div style="font-size:2em; text-align:center; margin-top:30px; margin-bottom:20px">Data Science Academy 7</div>
<hr>
<br>

<div style="font-size:4em; text-align:center; margin-bottom:30px; color:#00746E"><b>Vector AutoRegression</b></div>

In the VAR model, each variable is modeled as a <font color = "#28419b">linear combination of past values of itself and the past values of other variables </font> in the system. Since you have multiple time series that influence each other, it is modeled as a system of equations with one equation per variable (time series).

<a> <img src="https://www.machinelearningplus.com/wp-content/uploads/2019/07/Equation_VAR1_Model-min.png" width = "360"> </a>

Where, Y{1,t-1} and Y{2,t-1} are the first lag of time series Y1 and Y2 respectively.

The above equation is referred to as a VAR(1) model, because, each equation is of order 1, that is, it contains up to one lag of each of the predictors (Y1 and Y2).

Since the Y terms in the equations are interrelated, the Y’s are considered as endogenous variables, rather than as exogenous predictors.

Likewise, the second order VAR(2) model for two variables would include up to two lags for each variable (Y1 and Y2).

Credit: https://www.machinelearningplus.com/time-series/vector-autoregression-examples-python/

Second order VAR.

<a> <img src= "https://www.machinelearningplus.com/wp-content/uploads/2019/07/Equation_VAR2_Model-min.png?ezimgfmt=ng:webp/ngcb3"  ></a>

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import plotly.graph_objs as go # plotly graphical object
import plotly.express as px

# Import Statsmodels
from statsmodels.tsa.api import VAR
from statsmodels.tsa.stattools import adfuller
from statsmodels.tools.eval_measures import rmse, aic

In [3]:
## Let's use the store forecasting dataset.
df = pd.read_csv('../input/demand_store_forecast/train.csv')
buf = df[(df['store'] == 1 ) & (df['item'] == 1) | (df['store'] == 1 ) & (df['item'] == 2)].copy() # item 1&2 in store 1
buf = buf.set_index('date')
buf.index = pd.DatetimeIndex(buf.index).to_period('D')
# y = buf['sales']
# y_to_train = y.iloc[:(len(y)-365)]
# y_to_test = y.iloc[(len(y)-365):] # last year for testing

In [4]:
buf.item.value_counts()

2    1826
1    1826
Name: item, dtype: int64

In [5]:
buf.drop(['store'], axis = 1, inplace = True)

In [6]:
df = pd.read_csv('../input/demand_store_forecast/train.csv')
plot_df = df[(df['store'] == 1 ) & (df['item'] == 1) | (df['store'] == 1 ) & (df['item'] == 2)].copy() 
# Visualise time series
fig = px.line(plot_df, x= 'date', y= 'sales', color = 'item')
fig.show()


Both time series have fairly similar trend patterns over the years. 
Next step, check for Granger's Causality Test and Cointegration Test. 

## Granger's Causality Test
VAR - assumes each time series in the system influences each other. That is you can predict the series with past values of itself along with other series in the system. 

Granger’s causality tests the null hypothesis that the coefficients of past values in the regression equation is zero.


In simpler terms, the past values of time series (X) do not cause the other series (Y). So, if the p-value obtained from the test is lesser than the significance level of 0.05, then, you can safely reject the null hypothesis.

The below code implements the Granger’s Causality test for all possible combinations of the time series in a given dataframe and stores the p-values of each combination in the output matrix.

In [7]:
from statsmodels.tsa.stattools import grangercausalitytests
maxlag=12
test = 'ssr_chi2test'
def grangers_causation_matrix(data, variables, test='ssr_chi2test', verbose=False):    
    """Check Granger Causality of all possible combinations of the Time series.
    The rows are the response variable, columns are predictors. The values in the table 
    are the P-Values. P-Values lesser than the significance level (0.05), implies 
    the Null Hypothesis that the coefficients of the corresponding past values is 
    zero, that is, the X does not cause Y can be rejected.

    data      : pandas dataframe containing the time series variables
    variables : list containing names of the time series variables.
    """
    df = pd.DataFrame(np.zeros((len(variables), len(variables))), columns=variables, index=variables)
    for c in df.columns:
        for r in df.index:
            test_result = grangercausalitytests(data[[r, c]], maxlag=maxlag, verbose=False)
            p_values = [round(test_result[i+1][0][test][1],4) for i in range(maxlag)]
            if verbose: print(f'Y = {r}, X = {c}, P Values = {p_values}')
            min_p_value = np.min(p_values)
            df.loc[r, c] = min_p_value
    df.columns = [var + '_x' for var in variables]
    df.index = [var + '_y' for var in variables]
    return df


In [8]:
# Converting the time series data to long table format
wide_df = buf.pivot(columns = 'item')
wide_df = wide_df.reset_index(level = ['date'])
wide_df

date sales    
item                 1   2
0     2013-01-01    13  33
1     2013-01-02    11  43
2     2013-01-03    14  23
3     2013-01-04    13  18
4     2013-01-05    10  34
...          ...   ...  ..
1821  2017-12-27    14  55
1822  2017-12-28    19  50
1823  2017-12-29    15  50
1824  2017-12-30    27  56
1825  2017-12-31    23  67

[1826 rows x 3 columns]

In [9]:
wide_df.columns = wide_df.columns.droplevel(0)
wide_df
wide_df.columns = ['date', 'item1', 'item2']
wide_df.set_index('date', inplace = True)

In [10]:
grangers_causation_matrix(wide_df, variables = wide_df.columns)        

,item1_x,item2_x
item1_y,1.0,0.0
item2_y,0.0,1.0


If a given p-value is < significance level (0.05), then, the corresponding X series (column) causes the Y (row). In this case, the null hypothesis is significant (0.0), therefore we <u> reject null hypothesis, and say that `sales of item1` causes `sales of item2`.

This makes this system of multi time series a good candidate for using VAR models to forecast.

Next, let’s do the Cointegration test.

## Cointegration test
Cointegration test helps to establish the presence of a statistically significant connection between two or more time series.
Order of `integration(d)` is nothing but the number of differencing required to make a non-stationary time series stationary.


In [11]:
from statsmodels.tsa.vector_ar.vecm import coint_johansen

def cointegration_test(df, alpha=0.05): 
    """Perform Johanson's Cointegration Test and Report Summary"""
    out = coint_johansen(df,-1,5)
    d = {'0.90':0, '0.95':1, '0.99':2}
    traces = out.lr1
    cvts = out.cvt[:, d[str(1-alpha)]]
    def adjust(val, length= 6): return str(val).ljust(length)

    # Summary
    print('Name   ::  Test Stat > C(95%)    =>   Signif  \n', '--'*20)
    for col, trace, cvt in zip(df.columns, traces, cvts):
        print(adjust(col), ':: ', adjust(round(trace,2), 9), ">", adjust(cvt, 8), ' =>  ' , trace > cvt)


In [12]:
cointegration_test(wide_df)

Name   ::  Test Stat > C(95%)    =>   Signif  
 ----------------------------------------
item1  ::  260.48    > 12.3212   =>   True
item2  ::  1.14      > 4.1296    =>   False


## Split Train Test

In [13]:
nobs = 30
df_train, df_test = wide_df[0:-nobs], wide_df[-nobs:]

# Check size
print(df_train.shape)  # (1796, 2)
print(df_test.shape)  # (30, 2)

(1796, 2)
(30, 2)


## Check for Stationarity and Make the Time Series Stationary

How to test for stationarity?

In [14]:
def adfuller_test(series, signif=0.05, name='', verbose=False):
    """Perform ADFuller to test for Stationarity of given series and print report"""
    r = adfuller(series, autolag='AIC')
    output = {'test_statistic':round(r[0], 4), 'pvalue':round(r[1], 4), 'n_lags':round(r[2], 4), 'n_obs':r[3]}
    p_value = output['pvalue'] 
    def adjust(val, length= 6): return str(val).ljust(length)

    # Print Summary
    print(f'    Augmented Dickey-Fuller Test on "{name}"', "\n   ", '-'*47)
    print(f' Null Hypothesis: Data has unit root. Non-Stationary.')
    print(f' Significance Level    = {signif}')
    print(f' Test Statistic        = {output["test_statistic"]}')
    print(f' No. Lags Chosen       = {output["n_lags"]}')

    for key,val in r[4].items():
        print(f' Critical value {adjust(key)} = {round(val, 3)}')

    if p_value <= signif:
        print(f" => P-Value = {p_value}. Rejecting Null Hypothesis.")
        print(f" => Series is Stationary.")
    else:
        print(f" => P-Value = {p_value}. Weak evidence to reject the Null Hypothesis.")
        print(f" => Series is Non-Stationary.")    

In [15]:
# ADF Test on each column
for name, column in df_train.iteritems():
    adfuller_test(column, name=column.name)
    print('\n')

    Augmented Dickey-Fuller Test on "item1" 
    -----------------------------------------------
 Null Hypothesis: Data has unit root. Non-Stationary.
 Significance Level    = 0.05
 Test Statistic        = -3.0996
 No. Lags Chosen       = 23
 Critical value 1%     = -3.434
 Critical value 5%     = -2.863
 Critical value 10%    = -2.568
 => P-Value = 0.0266. Rejecting Null Hypothesis.
 => Series is Stationary.


    Augmented Dickey-Fuller Test on "item2" 
    -----------------------------------------------
 Null Hypothesis: Data has unit root. Non-Stationary.
 Significance Level    = 0.05
 Test Statistic        = -2.9131
 No. Lags Chosen       = 23
 Critical value 1%     = -3.434
 Critical value 5%     = -2.863
 Critical value 10%    = -2.568
 => P-Value = 0.0438. Rejecting Null Hypothesis.
 => Series is Stationary.




In [16]:
# Perform 1st order differencing 
df_differenced = df_train.diff().dropna()

In [17]:
# ADF Test on each column of 1st Differences Dataframe
for name, column in df_differenced.iteritems():
    adfuller_test(column, name=column.name)
    print('\n')

    Augmented Dickey-Fuller Test on "item1" 
    -----------------------------------------------
 Null Hypothesis: Data has unit root. Non-Stationary.
 Significance Level    = 0.05
 Test Statistic        = -12.6608
 No. Lags Chosen       = 22
 Critical value 1%     = -3.434
 Critical value 5%     = -2.863
 Critical value 10%    = -2.568
 => P-Value = 0.0. Rejecting Null Hypothesis.
 => Series is Stationary.


    Augmented Dickey-Fuller Test on "item2" 
    -----------------------------------------------
 Null Hypothesis: Data has unit root. Non-Stationary.
 Significance Level    = 0.05
 Test Statistic        = -10.7813
 No. Lags Chosen       = 25
 Critical value 1%     = -3.434
 Critical value 5%     = -2.863
 Critical value 10%    = -2.568
 => P-Value = 0.0. Rejecting Null Hypothesis.
 => Series is Stationary.




## How to select the Order (P) for the VAR Model?
- pick the order that gives least AIC. You can also compare for BIC, FPE, HQIC. 

In [18]:
model = VAR(df_differenced)
for i in list(range(1, 9)):
    result = model.fit(i)
    print('Lag Order =', i)
    print('AIC : ', result.aic)
    print('BIC : ', result.bic)
    print('FPE : ', result.fpe)
    print('HQIC: ', result.hqic, '\n')

Lag Order = 1
AIC :  8.593727174511944
BIC :  8.612095746560836
FPE :  5397.694412588351
HQIC:  8.600508866053561 

Lag Order = 2
AIC :  8.42020874739497
BIC :  8.4508369987815
FPE :  4537.850752032999
HQIC:  8.43151704034311 

Lag Order = 3
AIC :  8.309245169076634
BIC :  8.352144290901126
FPE :  4061.246594915047
HQIC:  8.32508445021831 

Lag Order = 4
AIC :  8.198019918371141
BIC :  8.253201118391852
FPE :  3633.748676197849
HQIC:  8.21839458120962 

Lag Order = 5
AIC :  8.095885242744536
BIC :  8.16335974541178
FPE :  3280.9410055816393
HQIC:  8.12079968751335 

Lag Order = 6
AIC :  7.648253862631674
BIC :  7.728032909122018
FPE :  2096.9818470929313
HQIC:  7.677712496308638 

Lag Order = 7
AIC :  7.640716955426182
BIC :  7.732811803676744
FPE :  2081.2370733459884
HQIC:  7.674724191747481 

Lag Order = 8
AIC :  7.633239054240262
BIC :  7.737660978983082
FPE :  2065.732578863938
HQIC:  7.671799313714544 



Based on the above output, we can select the order with minimum AIC : which is Lag order 6. Since lag order 7 and 8 is quite similar, we will go with Lag Order 6.  

Alternate methods to choose order(p) is use `model.select_order(maxlags)`.

In [19]:
x = model.select_order(maxlags=12)
x.summary()

,AIC,BIC,FPE,HQIC
0,9.049,9.055,8508.,9.051
1,8.600,8.619,5433.,8.607
2,8.428,8.459,4572.,8.439
3,8.317,8.360,4093.,8.333
4,8.205,8.260,3658.,8.225
5,8.102,8.170,3301.,8.127
6,7.653,7.733*,2108.,7.683
7,7.645,7.737,2090.,7.679
8,7.636,7.741,2072.,7.675
9,7.626,7.743,2051.,7.669


According to AIC, lag 12 is best order, but BIC has indicated lag 6. Since AIC, FPE and HQIC is lowest at lag 12, I will go with 12.

## Train the VAR Model of Selected Order (p)

In [20]:
model_fitted = model.fit(4)
model_fitted.summary()

  Summary of Regression Results   
Model:                         VAR
Method:                        OLS
Date:           Thu, 11, Mar, 2021
Time:                     12:47:08
--------------------------------------------------------------------
No. of Equations:         2.00000    BIC:                    8.25320
Nobs:                     1791.00    HQIC:                   8.21839
Log likelihood:          -12406.0    FPE:                    3633.75
AIC:                      8.19802    Det(Omega_mle):         3597.50
--------------------------------------------------------------------
Results for equation item1
              coefficient       std. error           t-stat            prob
---------------------------------------------------------------------------
const            0.007053         0.136503            0.052           0.959
L1.item1        -0.722904         0.025412          -28.447           0.000
L1.item2         0.065363         0.012777            5.116           0.000
L2.i

## Check for Serial Correlation of Residuals (Errors) using Durbin Watson Statistic

If there is any correlation left in the residuals, then, there is some pattern in the time series that is still left to be explained by the model. In that case, the typical course of action is to either increase the order of the model or induce more predictors into the system or look for a different algorithm to model the time series.

The value of this statistic can vary between 0 and 4. 
* Closer to the value 2, then there is no significant serial correlation
* Closer to 0, there is a positive serial correlation
* Closer to 4 implies negative serial correlation.

In [21]:
from statsmodels.stats.stattools import durbin_watson
out = durbin_watson(model_fitted.resid)

for col, val in zip(buf.columns, out):
    print(col, ':', round(val, 2))

item : 2.11
sales : 2.13


## How to Forecast VAR model using statsmodels

In [22]:
# Get the lag order
lag_order = model_fitted.k_ar
print(lag_order)  #> 4

4


In [23]:
# Input data for forecasting
forecast_input = df_differenced.values[-lag_order:]
forecast_input

array([[  4.,  -1.],
       [ 14.,   9.],
       [-13.,  14.],
       [  4., -24.]])

In [24]:
# Forecast
fc = model_fitted.forecast(y=forecast_input, steps=nobs)
df_forecast = pd.DataFrame(fc, index=wide_df.index[-nobs:], columns=wide_df.columns + '_1d')
df_forecast

,item1_1d,item2_1d
date,,
2017-12-02,-2.740655,5.716205
2017-12-03,0.966789,-1.179933
2017-12-04,2.106290,4.053257
2017-12-05,-1.492606,1.978590
2017-12-06,0.423257,-3.897174
2017-12-07,-0.614752,0.434992
2017-12-08,0.157846,-0.379618
2017-12-09,0.357833,1.066453
2017-12-10,-0.117954,0.362690


The forecasts are generated but it is on the scale of the training data used by the model. So, to bring it back up to its original scale, you need to de-difference it as many times you had differenced the original input data.

Invert the transformation to get the real forecast

In [25]:
def invert_transformation(df_train, df_forecast, second_diff=False):
    """Revert back the differencing to get the forecast to original scale."""
    df_fc = df_forecast.copy()
    columns = df_train.columns
    for col in columns:        
        # Roll back 2nd Diff
        if second_diff:
            df_fc[str(col)+'_1d'] = (df_train[col].iloc[-1]-df_train[col].iloc[-2]) + df_fc[str(col)+'_2d'].cumsum()
        # Roll back 1st Diff
        df_fc[str(col)+'_forecast'] = df_train[col].iloc[-1] + df_fc[str(col)+'_1d'].cumsum()
    return df_fc

In [26]:
df_results = invert_transformation(df_train, df_forecast, second_diff=False)   
df_results.loc[:, ['item1_forecast', 'item2_forecast']    ].round(2)

,item1_forecast,item2_forecast
date,,
2017-12-02,16.26,61.72
2017-12-03,17.23,60.54
2017-12-04,19.33,64.59
2017-12-05,17.84,66.57
2017-12-06,18.26,62.67
2017-12-07,17.65,63.11
2017-12-08,17.81,62.73
2017-12-09,18.16,63.79
2017-12-10,18.05,64.16


## Plot Forecast vs Actual

In [27]:
df_results_f = df_results.loc[:, ['item1_forecast', 'item2_forecast']    ].round(2)
df_results_f.set_index(df_test.index, inplace = True) 
df_results_f.reset_index(inplace = True)
df_results_f

,date,item1_forecast,item2_forecast
0,2017-12-02,16.26,61.72
1,2017-12-03,17.23,60.54
2,2017-12-04,19.33,64.59
3,2017-12-05,17.84,66.57
4,2017-12-06,18.26,62.67
5,2017-12-07,17.65,63.11
6,2017-12-08,17.81,62.73
7,2017-12-09,18.16,63.79
8,2017-12-10,18.05,64.16
9,2017-12-11,18.08,63.57


In [28]:
# Visualise time series
df_results_long = pd.melt(df_results_f, id_vars = ["date"], value_vars = ['item1_forecast', 'item2_forecast'], var_name = 'item', value_name ='sales')
df_results_long['item'] = df_results_long['item'].str.replace('_forecast', '')


In [29]:
df_results_long.head()

,date,item,sales
0,2017-12-02,item1,16.26
1,2017-12-03,item1,17.23
2,2017-12-04,item1,19.33
3,2017-12-05,item1,17.84
4,2017-12-06,item1,18.26


In [31]:
df_results_long['date'] = df_results_long.date.dt.strftime('%Y-%m-%d')

In [34]:
import plotly.express as px
fff = px.line(df_results_long, x = 'date', y = 'sales', color = 'item' )
fff.update_layout(title = 'Forecasted sales for Item 1 and Item 2')
fff.show()